In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [18]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install rouge_score --quiet

In [3]:
import os
import torch
import random
import json

from datasets import load_dataset, load_metric
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [4]:
model_name = "OjasPatil/intel-llama2-7b"

In [5]:
device_map = {"": 0}

In [6]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:81: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


config.json:   0%|          | 0.00/630 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [10]:
# Ignore warnings
logging.set_verbosity(logging.ERROR)
user_prompt = "What is Intel OpenVINO?"
prompt = f"<s>[INST]{user_prompt}[/INST]"
pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=100)

result = pipe(prompt)
print(result[0]['generated_text'].replace(prompt, ""))

OpenVINO is an open-source toolkit for converting Vue Intel and other models into render-invariant representations and settings the appropriate rendering environment. It enables efficient inference and rendering across diverse environments, including DLCs, clouds, and edge devices.  It supports various models, frameworks, and rendering environments and is available on GitHub.  More information can be found on the OpenVINO documentation page


In [11]:
def load_dataset_from_jsonl(file_path):
    dataset = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            entry = json.loads(line)
            dataset.append(entry)
    return dataset

In [12]:
dataset_file = "/content/rouge_test.jsonl"
dataset = load_dataset_from_jsonl(dataset_file)

In [27]:
def generate_response(user_prompt):
    prompt = f"<s>[INST]{user_prompt}[/INST]"
    pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=100)
    result = pipe(prompt)
    return result[0]['generated_text'].replace(prompt, "")

In [28]:
generated_responses = []
for entry in dataset:
    prompt = entry["question"]
    generated_response = generate_response(prompt)
    generated_responses.append(generated_response)

In [29]:
print(references[0])
print(generated_responses[0])

To receive Priority Support, you need to purchase and register a supported version of an Intel toolkit. Choose a Product: Intel® oneAPI Base Toolkit Intel® oneAPI Base & HPC Toolkit Intel® oneAPI Base & Rendering Toolkit
To receive Priority Support, you need to purchase a supported version of an Intel toolkit. Choose a Product: Intel® oneAPI Toolkit Intel® oneAPI Runtime Download Location: Linux* (various supported versions) and Windows* (various supported versions) and macOS* (various supported versions) Purchase a supported version of the Intel toolkit and get Prior


In [30]:
print(len(references))
print(len(generated_responses))

16
16


In [32]:
rouge_metric = load_metric("rouge")

references = [entry["answer"] for entry in dataset]
rouge_results = rouge_metric.compute(predictions=generated_responses, references=references)

In [33]:
print("\nROUGE Scores:")
print(f"ROUGE-1: {rouge_results['rouge1'].mid.fmeasure * 100:.2f}")
print(f"ROUGE-2: {rouge_results['rouge2'].mid.fmeasure * 100:.2f}")
print(f"ROUGE-L: {rouge_results['rougeL'].mid.fmeasure * 100:.2f}")


ROUGE Scores:
ROUGE-1: 35.23
ROUGE-2: 18.97
ROUGE-L: 28.82
